In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='vncns_inter_vs_local')
pre_to_mn_df.shape

## Create ordered type specific neurons

In [ ]:
All = slice(None)

In [ ]:
# segID	motor	has_soma	sensory	 neck	local	
m_r_tup = (All,True,All,All,All,All)
motor = pre_to_mn_df.loc[m_r_tup,:]
motor = motor.loc[motor.sum(axis=1).sort_values(ascending=False).index,:]
print(motor.shape)

### sensory neurons
# segID	motor	has_soma	sensory	 neck	local	
s_r_tup = (All,False,All,True,False,False)
sensory = pre_to_mn_df.loc[s_r_tup,:]
sensory = sensory.loc[sensory.sum(axis=1).sort_values(ascending=False).index,:]
print(sensory.shape)

### weird mn neurons
# segID	motor	has_soma	sensory	 neck	local	
w_r_tup = (All,False,True,True,False,False)
sensmns = pre_to_mn_df.loc[w_r_tup,:]
sensmns = sensmns.loc[sensmns.sum(axis=1).sort_values(ascending=False).index,:]
print(sensmns.shape)

### descending neurons
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True,All),:]
descending = descending.loc[descending.sum(axis=1).sort_values(ascending=False).index,:]
print(descending.shape)

### How many ascending neurons # 138
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
ascending = ascending.loc[ascending.sum(axis=1).sort_values(ascending=False).index,:]
print(ascending.shape)

### VNC neurons
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,All,All),:]
vncns = vncns.loc[vncns.sum(axis=1).sort_values(ascending=False).index,:]
print(vncns.shape)

fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
fragments.shape


In [ ]:
sensmns

## Separate according to local vs. intersegmental
Should some of these be local neurons instead?

In [ ]:

local = vncns.loc[(All,False,True,False,False,True,All),:]
print('local shape:{}'.format(local.shape))

# peel off some local neurons from the intersegmental neurons
intrseg = vncns.loc[(All,False,True,False,False,False,All),:]
print('inter shape:{}'.format(intrseg.shape))

index = intrseg.index.to_frame()
index.index = index.index.get_level_values('segID')
somenonT1 = index.non_T1_count<20
intrseg.loc[somenonT1.to_list(),:]

possible_local = intrseg.loc[somenonT1.to_list(),:]
possible_local.shape

# Order local_p by numbers of synapses
local_p = pd.concat([local,possible_local])
local_p = local_p.loc[local_p.sum(axis=1).sort_values(ascending=False).index,:]

intrseg_m = intrseg.loc[~somenonT1.to_numpy(),:]

print(intrseg_m.shape)


# Print the next tranche of neurons to see if any others should be typed and local
mi,arr = intrseg_m.index.sortlevel(level='non_T1_count')
mi_df = mi.to_frame()
# .loc[].segID
mi_df.loc[mi_df.non_T1_count<100].segID.to_list()
utils.save_df_csv(mi_df.loc[mi_df.non_T1_count<100],name='possible_local_from_intersegmental')

# Now just look for that giant 19B neuron and move it to the local stack
pmns = pd.DataFrame()
pmns['pt_position'] = [[52132, 120825, 1557]]

# looking to add this one particular neuron:
cv = cloudvolume.CloudVolume(client.info.get_datastack_info()['segmentation_source'], use_https=True)
segID = utils.segIDs_from_pts_service(pmns.pt_position,cv)

if any(pre_to_mn_df.index.get_level_values('segID').isin([648518346486760188])):
    print('Moving large neuron {} from intersegmental to local group'.format(segID))
    local_p = pd.concat([local_p,intrseg_m.loc[segID]])
    local_p = local_p.loc[local_p.sum(axis=1).sort_values(ascending=False).index,:]

locidx = local_p.index.to_frame()
# locidx.index = locidx.index.get_level_values('segID').to_list()
locidx.local = True
locmi = pd.MultiIndex.from_frame(locidx)
local_p.index = locmi
print(local_p.shape)

intrseg_m = intrseg_m.loc[mi_df.segID!=segID[0],:]
print('local shape:{} ({})'.format(local_p.shape,locidx.local.sum()))
print('inter shape:{}'.format(intrseg_m.shape))

In [ ]:
# pre_to_mn_df.loc[(648518346489708815)]
# # pre_to_mn_df
# any(pre_to_mn_df.index.get_level_values('segID').isin([648518346486760188]))

In [ ]:
# local = vncns.loc[(All,False,True,False,False,True,All),:]
# local.loc[(All,False,True,False,False,True,0),:]
# idx = local.index.to_frame()
# idx.loc[idx.non_T1_count!=0]

## Sort cell classes by similarity onto mns

### Descending neurons

In [ ]:
#print(descending.shape)
sim_mat = cosine_similarity(descending.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
descending_clustered = descending.iloc[clustered_order,:].copy()
descending_clustered.shape

### sensory neurons

In [ ]:
sim_mat = cosine_similarity(sensory.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
sensory_clustered = sensory.iloc[clustered_order,:].copy()
sensory_clustered.shape

### local neurons

In [ ]:
sim_mat = cosine_similarity(local_p.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
local_clustered = local_p.iloc[clustered_order,:].copy()
local_clustered.shape
local_clustered

### intersegmental neurons

In [ ]:
sim_mat = cosine_similarity(intrseg_m.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
intrseg_clustered = intrseg_m.iloc[clustered_order,:].copy()
intrseg_clustered.shape

### ascending neurons

In [ ]:
sim_mat = cosine_similarity(ascending.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
ascending_clustered = ascending.iloc[clustered_order,:].copy()
ascending_clustered.shape

In [ ]:
clustered_pMN_to_MN_df = pd.concat([
    motor,
    descending_clustered,
    sensory_clustered,
    intrseg_clustered,
    local_p, # local_clustered,
    ascending_clustered,
    fragments
])
print(clustered_pMN_to_MN_df.shape)

clustered_pMN_to_MN_df_no_frags = pd.concat([
    descending_clustered,
    sensory_clustered,
    intrseg_clustered,
    local_p, # local_clustered,
    ascending_clustered,
])
print(clustered_pMN_to_MN_df_no_frags.shape)

clustered_pMN_to_MN_df_with_clusteredvnc = pd.concat([
    descending_clustered,
    sensory_clustered,
    intrseg_clustered,
    local_clustered,
    ascending_clustered,
])
print(clustered_pMN_to_MN_df_with_clusteredvnc.shape)

# ordered_pMN_to_MN_df_no_desc = pd.concat([
#     sensory,
#     vncns,
#     ascending
# ])
# print(ordered_pMN_to_MN_no_desc_df.shape)


# ordered_pMN_to_MN_df_all = pd.concat([
#     descending,
#     sensory,
#     vncns,
#     ascending,
#     fragments
# ])
# print(ordered_pMN_to_MN_df_all.shape)


# Save the clustered_pMN_to_MN_df

In [ ]:
connectome_create.save_pre_to_mn_df(clustered_pMN_to_MN_df,ext='clustered_ordered')

In [ ]:
# Test
local_p = clustered_pMN_to_MN_df.loc[(All,False,True,False,False,True,All),:]
local_p.shape

# Show some clusters

In [ ]:
# All
# And plot the connectivity so that you can compare clusters
cmap = utils.white_dense()

mn_mi = clustered_pMN_to_MN_df_no_frags.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

reorder_mn_mat = np.log10(clustered_pMN_to_MN_df_no_frags.to_numpy()+1)
# reorder_mn_mat = np.log10(vncns.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(reorder_mn_mat, xticklabels=lbls,cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Local pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

## Local

In [ ]:
# Cluster according to everything but fragments
local_clustered = clustered_pMN_to_MN_df.loc[(All,False,True,False,False,True,All),:]
local_clustered = local_clustered.loc[local_clustered.sum(axis=1).sort_values(ascending=False).index,:]
# motor = motor.loc[motor.sum(axis=1).sort_values(ascending=False).index,:]

mn_mi = local_clustered.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')
sim_mat = cosine_similarity(local_clustered.to_numpy()) # pmn comparison, not transpose for mns

fig = plt.figure(1, figsize = [20,18])
ax = sns.heatmap(sim_mat)
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Local Neurons', fontsize =18)
plt.ylabel('Local Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

In [ ]:
# And plot the connectivity so that you can compare clusters
cmap = utils.white_dense()

mn_mi = local_clustered.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

reorder_mn_mat = np.log10(local_clustered.to_numpy()+1)
# reorder_mn_mat = np.log10(vncns.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(reorder_mn_mat, xticklabels=lbls,cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Local pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

## Intersegmental

In [ ]:
# Cluster according to everything but fragments
mn_mi = intrseg_clustered.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')
sim_mat = cosine_similarity(intrseg_clustered.to_numpy()) # pmn comparison, not transpose for mns

fig = plt.figure(1, figsize = [20,18])
ax = sns.heatmap(sim_mat)
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Intr Seg Neurons', fontsize =18)
plt.ylabel('Intr Seg Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

In [ ]:
# And plot the connectivity so that you can compare clusters
cmap = utils.white_dense()

mn_mi = intrseg_clustered.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

reorder_mn_mat = np.log10(intrseg_clustered.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(reorder_mn_mat,cmap=cmap)# xticklabels=lbls,)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Intersegmental pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

## Descending

In [ ]:
# Cluster according to everything but fragments
mn_mi = descending_clustered.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')
sim_mat = cosine_similarity(descending_clustered.to_numpy()) # pmn comparison, not transpose for mns

fig = plt.figure(1, figsize = [20,18])
ax = sns.heatmap(sim_mat)
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Descending Neurons', fontsize =18)
plt.ylabel('Descending Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

In [ ]:
mn_mi = descending_clustered.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

reorder_mn_mat = np.log10(descending_clustered.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(reorder_mn_mat, xticklabels=lbls,)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Descending pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()